<h1>Data Analysis after Scraping and Pre-Processing</h1>

In [28]:
import pandas as pd
DATA_PATH ='./data'
df_fighters = pd.read_csv(DATA_PATH+'/fighter_details.csv')
df_fights = pd.read_csv(DATA_PATH+'/total_fight_data.csv', sep=';')

In [29]:
df_fighters.head()

,fighter_name,Height,Weight,Reach,Stance,DOB
0,Tom Aaron,NaN,155 lbs.,NaN,NaN,"Jul 13, 1978"
1,Danny Abbadi,"5' 11""",155 lbs.,NaN,Orthodox,"Jul 03, 1983"
2,David Abbott,"6' 0""",265 lbs.,NaN,Switch,NaN
3,Shamil Abdurakhimov,"6' 3""",235 lbs.,"76""",Orthodox,"Sep 02, 1981"
4,Hiroyuki Abe,"5' 6""",145 lbs.,NaN,Orthodox,NaN


In [30]:
df_fights.head()

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,...,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Israel Adesanya,Yoel Romero,0,0,48 of 132,40 of 89,36%,44%,48 of 132,40 of 89,...,0 of 0,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Dan Miragliotta,"March 07, 2020","Las Vegas, Nevada, USA",UFC Middleweight Title Bout,Israel Adesanya
1,Weili Zhang,Joanna Jedrzejczyk,0,0,165 of 408,186 of 360,40%,51%,170 of 413,196 of 370,...,0 of 0,Decision - Split,5,5:00,5 Rnd (5-5-5-5-5),Keith Peterson,"March 07, 2020","Las Vegas, Nevada, USA",UFC Women's Strawweight Title Bout,Weili Zhang
2,Beneil Dariush,Drakkar Klose,1,0,12 of 20,15 of 25,60%,60%,17 of 28,27 of 37,...,0 of 0,KO/TKO,2,1:00,3 Rnd (5-5-5),Jason Herzog,"March 07, 2020","Las Vegas, Nevada, USA",Lightweight Bout,Beneil Dariush
3,Neil Magny,Jingliang Li,0,0,74 of 139,16 of 62,53%,25%,123 of 188,16 of 65,...,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,"March 07, 2020","Las Vegas, Nevada, USA",Welterweight Bout,Neil Magny
4,Alex Oliveira,Max Griffin,0,0,40 of 77,23 of 50,51%,46%,61 of 98,62 of 96,...,0 of 0,Decision - Split,3,5:00,3 Rnd (5-5-5),Mark Smith,"March 07, 2020","Las Vegas, Nevada, USA",Welterweight Bout,Alex Oliveira


In [31]:
df_fights.describe()

,R_KD,B_KD,R_SUB_ATT,B_SUB_ATT,R_PASS,B_PASS,R_REV,B_REV,last_round
count,5524.000000,5524.000000,5524.00000,5524.000000,5524.000000,5524.000000,5524.000000,5524.000000,5524.000000
mean,0.250181,0.179761,0.49294,0.351376,1.379073,0.823678,0.140478,0.134142,2.302136
std,0.521188,0.457599,0.93368,0.796855,2.271306,1.635999,0.425723,0.415524,1.003952
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,0.000000,0.000000,1.00000,0.000000,2.000000,1.000000,0.000000,0.000000,3.000000
max,5.000000,4.000000,10.00000,7.000000,26.000000,14.000000,5.000000,3.000000,5.000000
